# Notebook to predict the battery SOC using SSGP model

Dataset was collected from the following link:
https://www.kaggle.com/datasets/atechnohazard/battery-and-heating-data-in-real-driving-cycles

SSGP model was implemented using the following link:
https://github.com/linesd/SSGPR


In [ ]:
import sys
import numpy as np
from model.ssgpr import SSGPR
np.random.seed(1)  # set seed
import os.path
import pandas as pd
import chardet

In [ ]:
df_all = pd.DataFrame()
for root, dirs, files in os.walk("./data/Trips"):
    for name in files:
        filename = os.path.join(root, name)
        print(filename)
        df = pd.read_csv(filename,
                        sep=';',
                        encoding=chardet.detect(open(filename, 'rb').read())['encoding'])
        df['meas'] = name
        df_all = pd.concat([df_all, df])

In [ ]:
df_all

In [ ]:
df_all.to_pickle('data/df_all.pkl')

# 1. EDA

In [ ]:
df_all = pd.read_pickle('data/df_all.pkl')

In [ ]:
df_all.describe()

In [ ]:
# drop unnecessary signals
signals_to_drop = [
    'max. Battery Temperature [°C]',
    'displayed SoC [%]',
    'min. SoC [%]', 'max. SoC [%)',
    'Requested Coolant Temperature [°C]',
    'Unnamed: 23',
    'Velocity [km/h]]]',
    'Heating Power LIN [W]', # from here on, all signals are not available
    'Heater Voltage [V]',
    'Heater Current [A]',
    'Coolant Temperature Heatercore [°C]',
    'Coolant Temperature Inlet [°C]',
    'Ambient Temperature Sensor [°C]',
    'Coolant Volume Flow +500 [l/h]',
    'Temperature Coolant Heater Inlet [°C]',
    'Temperature Coolant Heater Outlet [°C]',
    'Temperature Heat Exchanger Outlet [°C]',
    'Temperature Defrost lateral left [°C]',
    'Temperature Defrost lateral right [°C]',
    'Temperature Defrost central [°C]',
    'Temperature Defrost central left [°C]',
    'Temperature Defrost central right [°C]',
    'Temperature Footweel Driver [°C]',
    'Temperature Footweel Co-Driver [°C]',
    'Temperature Feetvent Co-Driver [°C]',
    'Temperature Feetvent Driver [°C]',
    'Temperature Head Co-Driver [°C]',
    'Temperature Head Driver [°C]',
    'Temperature Vent right [°C]',
    'Temperature Vent central right [°C]',
    'Temperature Vent central left [°C]', 
    'Temperature Vent right [°C] ']
df_all = df_all.drop(columns=signals_to_drop)

In [ ]:
df_all.isna().sum()

In [ ]:
df_all.isna()

In [ ]:
# find measurements where nan values are present
df_all[df_all.isna()['SoC [%]']]['meas'].unique()

In [ ]:
meas_list = df_all[df_all.isna()['SoC [%]']]['meas'].unique()
for meas in meas_list:
    print(f"{meas}: {df_all[df_all['meas']==meas]['SoC [%]'].isna().sum()}")

In [ ]:
plt.figure()
plt.plot(df_all[df_all['meas']=='TripB34.csv']['SoC [%]'].isna())

In [ ]:
# nan are present only towards the end of the measurements. Therefore they can be dropped. No other strategy is needed.
df_all.dropna(inplace=True)

In [ ]:
df_all.describe()

In [ ]:
correlation_matrix = df_all.corr()

In [ ]:
correlation_matrix.loc['SoC [%]'].sort_values(ascending=False)

In [ ]:
# plot correlation matrix using plotly
import plotly.figure_factory as ff
import plotly.graph_objects as go

fig = ff.create_annotated_heatmap(z=correlation_matrix.values,
                                  x=list(correlation_matrix.columns),
                                  y=list(correlation_matrix.index),
                                  annotation_text=correlation_matrix.round(2).values,
                                  colorscale='Viridis')
fig.update_layout(
                title_text='Correlation Matrix',
                width = 1000,
                height = 1000,)
print(type(fig))
fig.show()

In [ ]:
from ydata_profiling import ProfileReport
profile = ProfileReport(df_all, title="Profiling Report")

In [ ]:
profile.to_widgets()

In [ ]:
profile.to_file("your_report.html")

# Train model

In [ ]:
# import train_test_split from sklearn.model_selection
from sklearn.model_selection import train_test_split

In [ ]:
output = 'SoC [%]'
X = df_all.drop(['SoC [%]', 'Time [s]', 'Motor Torque [Nm]', 'meas'], axis=1) # motor torque is colinear with acceleration
y = df_all[output]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# TODO: use seaborn jointplot to visualize the data


In [ ]:
X_train

In [ ]:
# create ssgpr instance
nbf = 100  # number of basis functions
ssgpr = SSGPR(num_basis_functions=nbf)
ssgpr.add_data(X_train.values, 
                y_train.values.reshape(-1,1), 
                X_test.values, 
                y_test.values.reshape(-1,1))
ssgpr.optimize(restarts=1, maxiter=10, verbose=True)

# predict on the test points
mu, sigma = ssgpr.predict(X_test.values, sample_posterior=False)

# evaluate the performance
NMSE, MNLP = ssgpr.evaluate_performance()
print("Normalised mean squared error (NMSE): %.5f" % NMSE)
print("Mean negative log probability (MNLP): %.5f" % MNLP)

In [ ]:
measurements = df_all['meas'].unique()
measurements

In [ ]:
for meas in measurements:
    df_meas = df_all.where(df_all['meas']==meas)
    time = df_meas['Time [s]']
    X = df_meas.drop(['SoC [%]', 'Time [s]', 'Motor Torque [Nm]', 'meas'], axis=1)
    y = df_meas['SoC [%]']
    plt.figure(figsize=(15,6))
    # predict on the measurement points
    mu, sigma = ssgpr.predict(X.values, sample_posterior=False)
    plt.plot(time, y, label="Measured data", color='blue')  # training data
    plt.plot(time, mu, label="Predictive mean", color='red')  # predictive mean
    plt.fill_between(time, (mu - 2 * sigma).flat, (mu + 2 * sigma).flat, color="#dddddd",
                        label="95% confidence interval")

    plt.grid()
    plt.xlabel("inputs, X")
    plt.ylabel("targets, Y")
    plt.legend(loc='lower right')
    plt.title(meas)
    plt.show()

In [ ]:
X = df_all.drop(['SoC [%]', 'Time [s]', 'Motor Torque [Nm]', 'meas'], axis=1)
y = df_all['SoC [%]']
time = range(len(df_all['Time [s]']))

In [ ]:
plt.figure(figsize=(15,6))
# predict on the measurement points
mu, sigma = ssgpr.predict(X.values, sample_posterior=False)
plt.plot(time, y, label="Training data", color='blue')  # training data
plt.plot(time, mu, label="Predictive mean", color='red')  # predictive mean
plt.fill_between(time, (mu - 2 * sigma).flat, (mu + 2 * sigma).flat, color="#dddddd",
                    label="95% confidence interval")

plt.grid()
plt.xlabel("inputs, X")
plt.ylabel("targets, Y")
plt.legend(loc='lower right')
plt.show()